# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.235117250637                   -0.50    8.0         
  2   -7.250190596770       -1.82       -1.39    1.0   9.12ms
  3   -7.251224163777       -2.99       -2.15    2.0   10.9ms
  4   -7.251290101294       -4.18       -2.25    3.0   13.2ms
  5   -7.251330274876       -4.40       -2.69    1.0   9.50ms
  6   -7.251337499824       -5.14       -3.08    1.0   9.38ms
  7   -7.251338724203       -5.91       -3.75    1.0   9.49ms
  8   -7.251338791564       -7.17       -4.18    2.0   11.1ms
  9   -7.251338798330       -8.17       -4.78    2.0   11.8ms
 10   -7.251338798688       -9.45       -5.39    2.0   11.8ms
 11   -7.251338798702      -10.84       -5.87    2.0   11.3ms
 12   -7.251338798704      -11.78       -6.19    1.0   9.26ms
 13   -7.251338798704      -12.24       -6.91    2.0   11.2ms
 14   -7.251338798705      -13.39       -7.40    3.0   12.9ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.08106952330203646
[ Info: Arnoldi iteration step 2: normres = 0.5924007478557886
[ Info: Arnoldi iteration step 3: normres = 0.6666319876176388
[ Info: Arnoldi iteration step 4: normres = 0.23194656987015
[ Info: Arnoldi iteration step 5: normres = 0.5101860899868604
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.33e-02, 6.59e-02, 3.76e-01, 3.37e-01, 3.02e-02)
[ Info: Arnoldi iteration step 6: normres = 0.3183513287366025
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (3.79e-03, 2.14e-01, 1.97e-01, 9.41e-02, 5.09e-02)
[ Info: Arnoldi iteration step 7: normres = 0.0650789564172651
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.22e-04, 1.36e-02, 8.04e-03, 2.95e-02, 5.20e-02)
[ Info: Arnoldi iteration step 8: normres = 0.12026901692848052
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (6.39e-06, 